# Fine-Tune CNN for Cats-Dogs Classification
### Fine-tune VGG16 top layers (conv block 5) and top-level fully connected classifier to classify images of cats and dogs.  
#### Adapted from fchollet/classifier_from_little_data_script_3.py (https://gist.github.com/fchollet/7eb39b44eb9e16e59632d25fb3119975) and blog https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [ ]:
import keras

In [ ]:
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

# import h5py
import numpy as np

In [ ]:
import tensorflow as tf
print (tf.__version__)
print (keras.__version__)

In [ ]:
# Set logging level
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
# Set random generator seed

seed = 123
np.random.seed(seed)

from tensorflow import set_random_seed
set_random_seed(246)

### Set location, number, and dimensions of images 

In [ ]:
# Location of images
train_data_dir = <<FILL-IN>>
validation_data_dir = <<FILL-IN>>

# Number of images
nb_train_samples = <<FILL-IN>>
nb_validation_samples = <<FILL-IN>>

# Image dimensions (150x150)
img_width, img_height = <<FILL-IN>>, <<FILL-IN>>

### Load pre-trained CNN

In [ ]:
# Load VGG16 network's imagenet weights, not including last fully connected block
base_model = applications.VGG16 (weights='imagenet', include_top=False, 
                            input_shape=(<<FILL-IN>>,img_height,3))
print ('Model loaded')
base_model.summary()

### Create classifier to put on top of CNN and load its weights (from features notebook)

In [ ]:
# Create fully connected layer as top model for CNN base
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))  # Convert 3D feature maps to 1D feature vectors
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# Load classifier's weights
top_model_weights = 'features_model-wts.h5'
top_model.load_weights (top_model_weights)

# Add classifier on top of CNN base
# model.add (top_model)
model = Model (inputs=base_model.input, outputs=top_model(base_model.output))

# Freeze weights in CNN up to last Conv block.  Set to FALSE to freeze weights
for layer in model.layers[:15]:
    layer.trainable = <<FILL-IN>>

# Compile model with SGD optimizer with momentum and very slow learning rate
model.compile(optimizer=optimizers.SGD (lr=1e-4, momentum=0.9),
              loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

### Prepare data

In [ ]:
# Set batch size
batch_size = 16

# Data augmentation setup
train_datagen = ImageDataGenerator (
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2, 
    horizontal_flip = True)
test_datagen = ImageDataGenerator (
    rescale = 1. / 255)

# Set up generator to read images found in subfolders of training data directory,
# and indefinitely generate batches of image data (scaled).  This is for training data.
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',    
    seed=seed)                # Set seed for reproducibility

# Set up generator to generate batched of validation data for model
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    seed=seed,
    shuffle=False)
 

In [ ]:
results = model.evaluate_generator(train_generator, steps=nb_train_samples // batch_size)
print (results)
results = model.evaluate_generator(<<FILL-IN>>, steps=nb_validation_samples // batch_size)
print (results)

### Fine tune model

In [ ]:
# Set number of training epochs.  Start with 5.
epochs = <<FILL-IN>>

# Train model, keeping track of history
from keras.callbacks import History
hist = model.fit_generator(
    train_generator, 
    steps_per_epoch = nb_train_samples // batch_size,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    initial_epoch=0,
    verbose = 2)

In [ ]:
results = model.evaluate_generator(<<FILL-IN>>, steps=nb_train_samples // batch_size)
print (results)
results = model.evaluate_generator(validation_generator, steps=nb_validation_samples // batch_size)
print (results)

### Save model and weights

In [ ]:
# Save model & weights to HDF5 file
model_file = '<<FILL-IN>>' 
model.save(model_file + '.h5')

# Save model to JSON file & weights to HDF5 file
model_json = model.to_json()
with open(model_file + '.json','w') as json_file:
    json_file.write(model_json)
model.save_weights(model_file+'-wts.h5')

# Results on validation set
print (model.metrics_names)
results = model.evaluate_generator(validation_generator, steps=nb_validation_samples // batch_size)
print (results)

### Load model again and test

In [ ]:
model2 = keras.models.load_model(model_file+'.h5')

print (model2.metrics_names)
results = model.evaluate_generator(validation_generator, steps=nb_validation_samples // batch_size)
print (results)
# scores = model2.evaluate_generator(validation_generator)

### Print training history

In [ ]:
print (hist.history)

### Plot performance metrics

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

result = hist.history

# summarize history for accuracy
plt.plot(result['acc'])
plt.plot(result['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()